In [1]:
import pickle as pkl
import numpy as np

path = "."
filename = "crowd_dict.pkl"

In [9]:
with open("%s/%s"%(path,filename),"rb") as _in:
    similarity_dataset = pkl.load(_in)


In [ ]:
"""
Author's Note:
This code aligns the textual description with the video clips. In order to get the textual
  description corresponding to key #5 in the similarity dataset, simply call phrase_lookup[5]
"""

import pandas as pd # Pandas library enables data manipulation
data_url = "./revisedBDDX.csv"
def load_bddx_data(csv_name):
    column_names = ['Index', 'InputVideo', '1S', '1E', '1A', '1J', '2S', '2E', '2A', '2J', '3S', '3E', '3A', '3J',
                    '4S', '4E', '4A', '4J','5S', '5E', '5A', '5J','6S', '6E', '6A', '6J','7S', '7E', '7A', '7J',
                    '8S', '8E', '8A', '8J','9S', '9E', '9A', '9J','10S', '10E', '10A', '10J','11S', '11E', '11A', '11J',
                    '12S', '12E', '12A', '12J','13S', '13E', '13A', '13J','14S', '14E', '14A', '14J','15S', '15E', '15A', '15J']
    
    return pd.read_csv(csv_name, names=column_names)

bddx = load_bddx_data(data_url)
bddx = bddx.drop(['1S', '1E','2S', '2E','3S', '3E','4S', '4E','5S', '5E','6S', '6E','7S', '7E','8S', '8E','9S', '9E','10S', '10E','11S', '11E','12S', '12E','13S', '13E','14S', '14E','15S', '15E', ], axis=1)
bddx = bddx.fillna("")

bddx['1AJ'] = bddx[['1A', '1J']].agg(' '.join, axis=1)
bddx['2AJ'] = bddx[['2A', '2J']].agg(' '.join, axis=1)
bddx['3AJ'] = bddx[['3A', '3J']].agg(' '.join, axis=1)
bddx['4AJ'] = bddx[['4A', '4J']].agg(' '.join, axis=1)
bddx['5AJ'] = bddx[['5A', '5J']].agg(' '.join, axis=1)
bddx['6AJ'] = bddx[['6A', '6J']].agg(' '.join, axis=1)
bddx['7AJ'] = bddx[['7A', '7J']].agg(' '.join, axis=1)
bddx['8AJ'] = bddx[['8A', '8J']].agg(' '.join, axis=1)
bddx['9AJ'] = bddx[['9A', '9J']].agg(' '.join, axis=1)
bddx['10AJ'] = bddx[['10A', '10J']].agg(' '.join, axis=1)
bddx['11AJ'] = bddx[['11A', '11J']].agg(' '.join, axis=1)
bddx['12AJ'] = bddx[['12A', '12J']].agg(' '.join, axis=1)
bddx['13AJ'] = bddx[['13A', '13J']].agg(' '.join, axis=1)
bddx['14AJ'] = bddx[['14A', '14J']].agg(' '.join, axis=1)
bddx['15AJ'] = bddx[['15A', '15J']].agg(' '.join, axis=1)

bddx = bddx.drop(['Index', '1A', '1J', '2A', '2J', '3A', '3J', '4A', '4J', '5A', '5J', '6A', '6J', '7A', '7J', '8A', '8J', '9A', '9J', '10A', '10J', '11A', '11J', '12A', '12J', '13A', '13J', '14A', '14J', '15A', '15J', ], axis=1)
bddx = bddx.drop(bddx.index[0])

sdlList = []
for index, row in bddx.iterrows():
    sdlList.append(row.astype(str))
    
sdlStatements = []
for i in range(len(sdlList)):
    sdlStatements.append({'1': sdlList[i]['1AJ'], '2': sdlList[i]['2AJ'], '3': sdlList[i]['3AJ'], '4': sdlList[i]['4AJ'], 
                        '5': sdlList[i]['5AJ'], '6': sdlList[i]['6AJ'], '7': sdlList[i]['7AJ'], '8': sdlList[i]['8AJ'], 
                        '9': sdlList[i]['9AJ'], '10': sdlList[i]['10AJ'], '11': sdlList[i]['11AJ'], '12': sdlList[i]['12AJ'], 
                        '13': sdlList[i]['13AJ'], '14': sdlList[i]['14AJ'], '15': sdlList[i]['15AJ']})

phrase_lookup = []

for i in range(len(sdlStatements)):
    keysTemp = list(sdlStatements[i].keys())
    for j in keysTemp:
        if (sdlStatements[i][j]==" "):
            continue
        else:
            phrase_lookup.append(sdlStatements[i][j])


In [15]:
# TODO: This is where metric stuff will go
def read_embeddings(embeddings_path):
    """Arguments:
        - embeddings_path: path to the embeddings
    """
    with open(embeddings_path, 'r') as in_stream:
        embeddings = []
        for line in in_stream:
            line = '['+line.replace(' ',',')+']'
            embeddings.append(eval(line))
        return embeddings
    return []

embeddings = np.array(read_embeddings("./bddx_embeddings.txt"))

def sent2vec_metric(a,b): # a and b should be indices taken from similarity_dataset
    return np.linalg.norm(embeddings[a]-embeddings[b]) # Euclidean Distance


In [31]:
"""
Author's Note:
How to change this code:
  Simply change "metric = sent2vec_metric" to a different metric function. The metric function
  should take as input two indices taken from the similarity_dataset.
How to read metric_output:
  For a given base index (e.g., 15096), it shows which ranking each video has under the metric
  e.g., [1,0,2,5,3,4] mean that the second video (index 23555) is the most similar, and the
  fourth video (index 17262) is the least similar.
"""

metric_output = {}
metric = sent2vec_metric # This should be a function that takes two indices as inputs

for base in similarity_dataset:
    metric_output[base] = [0]*len(similarity_dataset[base])
    output = []
    for comparison in similarity_dataset[base]:
        output.append( metric(base,comparison) )
    order = np.argsort(output)
    for i,sort_order in enumerate(order):
        metric_output[base][sort_order] = i


In [32]:
brk = 5
for key in metric_output:
    print(metric_output[key])
    print(similarity_dataset[key])
    brk-=1
    if brk <= 0:
        break
    

[1, 0, 2, 5, 3, 4]
[14293, 23555, 2079, 24844, 17262, 15380]
[5, 0, 3, 1, 2, 4]
[19200, 15559, 2659, 11364, 898, 18227]
[4, 2, 5, 1, 0, 3]
[1399, 5208, 10498, 9388, 17117, 1904]
[3, 1, 0, 4, 5, 2]
[738, 14371, 5451, 24023, 26526, 13481]
[5, 1, 3, 2, 4, 0]
[18824, 13899, 24471, 2071, 2451, 8295]
